# Python Data Model
Python数据模型可以说是对Python内部运行机制的一种描述, 大多讲述的是一些魔术方法(双下函数)  
通过魔术方法, 可以来模拟Python中的基础语言结构:
* Iteration 迭代
* Collection 集合
* Attribute access 存取属性
* Operator overloading 操作符重载
* Function and method invocation 函数调用
* Object creation and destruction 对象生成与析构
* String representation and formatting 字符串表达与格式化
* Managed contexts(e.g. `with`语法)

### e.g. 1-1 

In [32]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades,diamonds,clubs,hearts'.split(',')
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]

In [33]:
deck = FrenchDeck()
len(deck)

52

In [14]:
deck[0], deck[1]

(Card(rank='2', suit='spades'), Card(rank='3', suit='spades'))

Pick a random card

In [15]:
from random import choice

In [16]:
choice(deck)
choice(deck)

Card(rank='9', suit='diamonds')

Card(rank='3', suit='spades')

使用魔术方法的优点
* 对于用户可以更容易的使用, 对于容器不需要记住特定的方法, e.g. `.size()`? `.length()`?
* 对容易的使用Python标准库, 不用重复造轮子 e.g. `random.choice()`

在`__getitem__`中代理了`._cards`, 所以也可以支持切片(slicing)

In [17]:
deck[:3]
deck[12::13]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

只要实现了`__getitem__`了, 那么Iteration

**Note** 实现getitem, 如何实现Iteration  
只实现`__getitem__`, `for`循环是如何工作的,
在下面的实现中类似于下面代码

In [38]:
def increasing():
    index = 0
    while True:
        yield index
        index += 1
        
def iterable_wrapper(func):
    for index in increasing():
        try:
            card = deck.__getitem__(index)
            yield card
        except IndexError or StopIteration:
            break

for card in iterable_wrapper(deck):
    pass

In [35]:
for card in deck:
    # doing something
    pass

同样, 也支持`reversed()`

In [36]:
for card in reversed(deck):
    print(card)

Card(rank='A', suit='hearts')
Card(rank='K', suit='hearts')
Card(rank='Q', suit='hearts')
Card(rank='J', suit='hearts')
Card(rank='10', suit='hearts')
Card(rank='9', suit='hearts')
Card(rank='8', suit='hearts')
Card(rank='7', suit='hearts')
Card(rank='6', suit='hearts')
Card(rank='5', suit='hearts')
Card(rank='4', suit='hearts')
Card(rank='3', suit='hearts')
Card(rank='2', suit='hearts')
Card(rank='A', suit='clubs')
Card(rank='K', suit='clubs')
Card(rank='Q', suit='clubs')
Card(rank='J', suit='clubs')
Card(rank='10', suit='clubs')
Card(rank='9', suit='clubs')
Card(rank='8', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='2', suit='clubs')
Card(rank='A', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(r

**Note** 对于`reversed()`的实现类似于以下代码, 过程中调用了`__len__()`

In [39]:
def self_reversed(obj):
    l = len(obj)
    for index in range(l-1, -1, -1):
        item = obj.__getitem__(index)
        # do something...

并且, `FrenchDeck`作为Iteration, 尽管没有实现 `__contains__`方法, 
也可以实现`in`操作符

In [40]:
Card('Q', 'hearts') in deck

True

**Note** `__contains__`是如何实现的:
因为`FrenchDeck`作为`Iteration`, 所以可以遍历实现, 类似下面的代码

In [41]:
def contains(other_item):
    for item in deck:
        if item == other_item:
            return True
    return False

## 何时使用魔术方法
首先要知道, 魔术方法主演是由解释器调用而不是你调用, 比如你应该使用`len(my_obj)`而不是`my_obj.__len__()`  
其次, 使用内建方法一般比显式调用魔术方法速度要快. 在CPython中, 解释器对于部分的并且常用的内建方法要快, 经过了特制的优化